In [1]:
from  train_highway_env import *
from bokeh.io import output_notebook
from bokeh.plotting import show
import rlrom.plots
output_notebook()  # Add this at the top of your notebook

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2025-06-07 18:34:21.545816: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-07 18:34:21.553063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-07 18:34:21.561569: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Atte

Loading BokehJS ...

In [82]:

cfg = utils.load_cfg('cfg_main.yml')    
tester = HwEnvTester(cfg)
tester.run_seed(seed=0, num_steps=100)

lay = """            
    ego_vx
    car1_in_front
    car2_in_front
    sat(safe)
    sat(safe_too_long)
    sat(phi_term)
    """
fig, status=  rlrom.plots.get_fig(tester.env, lay)
show(fig)

loading field [ cfg_env ] from file [ cfg_env.yml ]
loading field [ cfg_train ] from file [ cfg_train.yml ]
loading field [ cfg_specs ] from file [ cfg_specs.yml ]
INFO: manual_control set to True, stay alert.
Crash
ego_vx
car1_in_front
car2_in_front
sat(safe)
sat(safe_too_long)
sat(phi_term)


In [33]:
tester.env.close()